In [ ]:
## IMPORTS
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, \
                            roc_auc_score, precision_score, f1_score, recall_score, \
                            mean_squared_error, mean_absolute_error, max_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import sys

In [ ]:
# change flag to run local or in colab
colab_env = False

results_dir = ""
model = "svm"
if colab_env:
    # default to run in colab
    from google.colab import drive
    drive.mount('/content/drive')

    sys.path.insert(1, "/content/drive/Shareddrives/tcc_pos/utils")
    import model_utils

    results_dir = "/content/drive/Shareddrives/tcc_pos/results/"
    dataframe_path = "/content/drive/Shareddrives/tcc_pos/datasets/no_filtered_analysis.csv"   
else:
    # default to run local
    dataframe_path = "../../pre_processing/datasets/preprocessed_data/no_filtered_analysis.csv"

    sys.path.insert(1, "../utils/")
    import model_utils

    results_dir = "./"

In [ ]:
df = pd.read_csv(dataframe_path, sep = ",")

In [ ]:
# Drop da coluna de RA
if "RA" in df.columns:
    print("Removing column RA")
    df.drop('RA', inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
# Run all predictions based on fold n_splits
def run_prediction(X_train, X_test, y_train, y_test, splits):

    # used to evaluate best model params
    gridsearch_metrics = {
        'accuracy': 'accuracy',
        'rec':'recall', 
        'auc':'roc_auc', 
        'f1': 'f1',
        'precision': 'precision',
        'mse': 'neg_mean_squared_error',
        'mae': 'neg_mean_absolute_error'
    }

    # model gridsearch params
    model_params = {
        'clf__kernel': ['linear','rbf', 'sigmoid'],
        'clf__C': [0.3,0.5,1,2,3,10],
        'clf__gamma': [0.3,0.5,1],
        'clf__decision_function_shape':['ovo', 'ovr'],
        'clf__shrinking': [True, False]
    }
    
    ## Estimator Standard Scaler
    estimator = [('scl', StandardScaler()), ('clf', SVC())]
    model_pipeline = Pipeline(estimator)
    
    # gridsearch initialization
    clf = GridSearchCV(model_pipeline,
                       param_grid = model_params,
                       n_jobs= 10,
                       cv = StratifiedKFold(n_splits=splits),
                       verbose= 1,
                       # scoring = gridsearch_metrics,
                       refit='accuracy')

    clf.fit(X_train, y_train)
    # print(clf.best_params_)
    
    predictions = clf.predict(X_test)

    predict_data = {
        "n_splits": splits,
        "confusion_matrix": confusion_matrix(y_test, predictions),
        "classification_report": classification_report(y_test, predictions),
        "model_accuracy": accuracy_score(y_test, predictions),
        "f1_score": f1_score(y_test, predictions),
        "precision_score": precision_score(y_test, predictions),
        "roc_score": roc_auc_score(y_test, predictions),
        "recall_score": recall_score(y_test, predictions),
        "epoch_params": clf.best_params_,
        "max_error": max_error(y_test, predictions),
        "mean_squared_error": mean_squared_error(y_test, predictions),
        "mean_absolute_error": mean_absolute_error(y_test, predictions)
    }

    return predict_data

In [ ]:
def main():
    X = df.iloc[:, df.columns != "EVADIDO"].values
    y = df["EVADIDO"]

    all_predictions = []
    max_splits = 10
    n_runs = 2
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify = y)


    for run in range(n_runs):
        print(f"Running {run+1} of {n_runs} \n\n")
        for split in range(2, max_splits, 2):
            print(f"\nStratifiedKFold config: {split} \n")
        
            # return best k element
            all_predictions.append(run_prediction(X_train, X_test, y_train, y_test, split))

        # General model outputs
        model_utils.generate_output(all_predictions, results_dir, model)
        print(f"############################ FINISHED RUN {run+1} ############################")
        
    # CSV Output results
    model_utils.generate_output_csv(all_predictions, results_dir, model)
    print(f"############################ FINISHED ALL ############################")

In [ ]:
if __name__ == "__main__":
    main()